In [263]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_log_error
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess, Fourier

# Time Related Features
def create_date_features(df):
    df['month'] = df.index.month.astype("int8")
    df['day_of_month'] = df.index.day.astype("int8")
    df['day_of_year'] = df.index.dayofyear.astype("int16")
    df['week_of_year'] = (df.index.isocalendar().week).astype("int8")
    df['day_of_week'] = (df.index.dayofweek + 1).astype("int8")
    df['year'] = df.index.year.astype("int32")
    df["is_wknd"] = (df.index.weekday // 4).astype("int8")
#    df["quarter"] = df.index.quarter.astype("int8")
#    df['is_month_start'] = df.index.is_month_start.astype("int8")
#    df['is_month_end'] = df.index.is_month_end.astype("int8")
#    df['is_quarter_start'] = df.index.is_quarter_start.astype("int8")
#    df['is_quarter_end'] = df.index.is_quarter_end.astype("int8")
    df['is_year_start'] = df.index.is_year_start.astype("int8")
    df['is_year_end'] = df.index.is_year_end.astype("int8")
    # 0: Winter - 1: Spring - 2: Summer - 3: Fall
    df["season"] = np.where(df.month.isin([12,1,2]), 0, 1)
    df["season"] = np.where(df.month.isin([6,7,8]), 2, df["season"])
    df["season"] = np.where(df.month.isin([9, 10, 11]), 3, df["season"]).astype("int8")
    return df



#seasonal plot functie
def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax

#periodogram laat zien in hoeverre er seasonality in de dataset zit
def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax


def lagplot(x, y=None, lag=1, standardize=False, ax=None, **kwargs):
    from matplotlib.offsetbox import AnchoredText
    x_ = x.shift(lag)
    if standardize:
        x_ = (x_ - x_.mean()) / x_.std()
    if y is not None:
        y_ = (y - y.mean()) / y.std() if standardize else y
    else:
        y_ = x
    corr = y_.corr(x_)
    if ax is None:
        fig, ax = plt.subplots()
    scatter_kws = dict(
        alpha=0.75,
        s=3,
    )
    line_kws = dict(color='C3', )
    ax = sns.regplot(x=x_,
                     y=y_,
                     scatter_kws=scatter_kws,
                     line_kws=line_kws,
                     lowess=True,
                     ax=ax,
                     **kwargs)
    at = AnchoredText(
        f"{corr:.2f}",
        prop=dict(size="large"),
        frameon=True,
        loc="upper left",
    )
    at.patch.set_boxstyle("square, pad=0.0")
    ax.add_artist(at)
    ax.set(title=f"Lag {lag}", xlabel=x_.name, ylabel=y_.name)
    return ax


def plot_lags(x, y=None, lags=6, nrows=1, lagplot_kwargs={}, **kwargs):
    import math
    kwargs.setdefault('nrows', nrows)
    kwargs.setdefault('ncols', math.ceil(lags / nrows))
    kwargs.setdefault('figsize', (kwargs['ncols'] * 2, nrows * 2 + 0.5))
    fig, axs = plt.subplots(sharex=True, sharey=True, squeeze=False, **kwargs)
    for ax, k in zip(fig.get_axes(), range(kwargs['nrows'] * kwargs['ncols'])):
        if k + 1 <= lags:
            ax = lagplot(x, y, lag=k + 1, ax=ax, **lagplot_kwargs)
            ax.set_title(f"Lag {k + 1}", fontdict=dict(fontsize=14))
            ax.set(xlabel="", ylabel="")
        else:
            ax.axis('off')
    plt.setp(axs[-1, :], xlabel=x.name)
    plt.setp(axs[:, 0], ylabel=y.name if y is not None else x.name)
    fig.tight_layout(w_pad=0.1, h_pad=0.1)
    return fig


def make_lags(ts, lags):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(1, lags + 1)
        },
        axis=1)





In [264]:
train = pd.read_csv('data/train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)
train.date = train.date.dt.to_period('D')
train = train.set_index(['store_nbr', 'family', 'date']).sort_index()



test = pd.read_csv("data/test.csv", parse_dates=['date'], infer_datetime_format=True)
transactions = pd.read_csv("data/transactions.csv", parse_dates=['date'], infer_datetime_format=True)
stores = pd.read_csv("data/stores.csv",index_col='store_nbr')
oil = pd.read_csv("data/oil.csv", parse_dates=['date'], infer_datetime_format=True, index_col='date')
holidays_events = pd.read_csv("data/holidays_events.csv", parse_dates=['date'], infer_datetime_format=True)

#foutje uit ander notebook meenemen
holidays_events['date'] = holidays_events['date'].replace({'2013-04-29' : 
                                         pd.to_datetime('2013-03-29')})




In [265]:
#Dataframe maken waarin je events in tijd kwijt kan
#let op dit is de train periode EN de test periode
calendar = pd.DataFrame(index=pd.date_range('2013-01-01','2017-08-31'))
#olieprijs toevoegen
#calendar = calendar.join(oil, how='left')
#calendar = calendar.rename(columns={"dcoilwtico": "oilprice"})
#lege waarden vullen
#calendar['oilprice'].fillna(method='ffill', inplace=True)
#eerste waarde vullen
#calendar['oilprice'].fillna(method='bfill', inplace=True)


data_oil = pd.read_csv('data/oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')
data_oil['ma_oil'] = data_oil['dcoilwtico'].rolling(7).mean()

calendar = calendar.merge(data_oil, how='left', left_index=True, right_index=True)
calendar['ma_oil'].fillna(method='ffill', inplace=True)









#tijdsgebonden features
calendar = create_date_features(calendar)



In [266]:
special_event = holidays_events['date'][holidays_events['type']=='Event']
additional_day = holidays_events['date'][(holidays_events['type']=='Additional') & (holidays_events['locale'] == 'National')]

national_transferred= holidays_events['date'][(holidays_events['type'] == 'Transfer') & (holidays_events['locale'] == 'National')]
national_bridged = holidays_events['date'][(holidays_events['type']=='Bridge')]
national_Workday = holidays_events['date'][(holidays_events['type']=='Work Day')]

national_holiday = holidays_events['date'][(holidays_events['type'] == 'Holiday') & (holidays_events['locale'] == 'National') & (holidays_events['transferred'] == False)]



local_transferred = holidays_events[(holidays_events['type'] == 'Transfer') & (holidays_events['locale'] == 'Local')]
local_transferred['combined'] = local_transferred[['locale', 'locale_name']].agg('-'.join, axis=1)
local_transferred.index = local_transferred.date
local_transferred = local_transferred[['combined']]

local_holiday = holidays_events[(holidays_events['type'] == 'Holiday') & (holidays_events['locale'] == 'Local') & (holidays_events['transferred'] == False)]
local_holiday['combined'] = local_holiday[['locale', 'locale_name']].agg('-'.join, axis=1)
local_holiday.index = local_holiday.date
local_holiday = local_holiday[['combined']]

regional_holiday = holidays_events[(holidays_events['type'] == 'Holiday') & (holidays_events['locale'] == 'Regional') & (holidays_events['transferred'] == False)]
regional_holiday['combined'] = regional_holiday[['locale', 'locale_name']].agg('-'.join, axis=1)
regional_holiday.index = regional_holiday.date
regional_holiday = regional_holiday[['combined']]


C:\Users\Tijs\AppData\Local\Temp/ipykernel_13532/1075823077.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  local_transferred['combined'] = local_transferred[['locale', 'locale_name']].agg('-'.join, axis=1)
C:\Users\Tijs\AppData\Local\Temp/ipykernel_13532/1075823077.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  local_holiday['combined'] = local_holiday[['locale', 'locale_name']].agg('-'.join, axis=1)
C:\Users\Tijs\AppData\Local\Temp/ipykernel_13532/1075823077.py:23: SettingWithCopyWarning: 
A va

In [267]:
sp = pd.DataFrame(calendar.index.isin(special_event.tolist()))
sp.index = calendar.index
calendar['specialevent'] = sp

ad = pd.DataFrame(calendar.index.isin(additional_day.tolist()))
ad.index = calendar.index
calendar['additionalday'] = ad

nt = pd.DataFrame(calendar.index.isin(national_transferred.tolist()))
nt.index = calendar.index
calendar['nationaltransferred'] = nt

nb = pd.DataFrame(calendar.index.isin(national_bridged.tolist()))
nb.index = calendar.index
calendar['nationalbridged'] = nb

wd = pd.DataFrame(calendar.index.isin(national_Workday.tolist()))
wd.index = calendar.index
calendar['nationalworkday'] = wd

nh = pd.DataFrame(calendar.index.isin(national_holiday.tolist()))
nh.index = calendar.index
calendar['nationalholiday'] = nh

In [268]:
#onehotting the local ones
calendar = calendar.join(
    pd.get_dummies(local_transferred,prefix='local_transferred')
    , how='left'
    )

calendar = calendar.join(
    pd.get_dummies(local_holiday,prefix='local_holiday').groupby('date').sum()
    , how='left'
    )

calendar = calendar.join(
    pd.get_dummies(regional_holiday,prefix='regional_holiday')
    , how='left'
    )
calendar = calendar.fillna(0)

#promotion avgs
calendar['promotion'] = train[['onpromotion']].groupby('date').sum()
train = train.drop(columns='onpromotion')

In [269]:
end_date='2017-08-15'
start_date='2017-04-01'

In [270]:
y = train.unstack(['store_nbr', 'family']).loc[start_date:end_date]

In [271]:
fourier = CalendarFourier(freq='W', order=4)
dp = DeterministicProcess(index=y.index,
                          constant=False,
                          order=1,
                          seasonal=False,
                          additional_terms=[fourier],
                          drop=True)
X = dp.in_sample()

In [272]:
calendar.index = calendar.index.to_period('D')

In [273]:
calendar['promotion'] = calendar['promotion'].fillna(0)

In [274]:
Xfinal = X.join(calendar)

In [275]:
model = Ridge(fit_intercept=True, solver='auto', alpha=0.5, normalize=True) # try alpha,0.1 ,0.3 ,0.6 and 0.9
model.fit(Xfinal, y)
y_pred = pd.DataFrame(model.predict(Xfinal), index=Xfinal.index, columns=y.columns)

C:\Users\Tijs\anaconda3\envs\TS\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


In [276]:
y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()
y_target['sales_pred'] = y_pred['sales'].clip(0.) 
y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred']))

family
AUTOMOTIVE                   0.24
BABY CARE                    0.06
BEAUTY                       0.24
BEVERAGES                    0.18
BOOKS                        0.02
BREAD/BAKERY                 0.12
CELEBRATION                  0.28
CLEANING                     0.19
DAIRY                        0.13
DELI                         0.10
EGGS                         0.14
FROZEN FOODS                 0.13
GROCERY I                    0.20
GROCERY II                   0.33
HARDWARE                     0.25
HOME AND KITCHEN I           0.24
HOME AND KITCHEN II          0.20
HOME APPLIANCES              0.14
HOME CARE                    0.11
LADIESWEAR                   0.24
LAWN AND GARDEN              0.20
LINGERIE                     0.37
LIQUOR,WINE,BEER             0.54
MAGAZINES                    0.23
MEATS                        0.11
PERSONAL CARE                0.12
PET SUPPLIES                 0.20
PLAYERS AND ELECTRONICS      0.20
POULTRY                      0.11
PREPARE

In [277]:
from joblib import Parallel, delayed
import warnings

from sklearn.linear_model import Ridge
from sklearn.ensemble     import RandomForestRegressor

class CustomRegressor():
    
    def __init__(self, n_jobs=-1, verbose=0):
        
        self.n_jobs = n_jobs
        self.verbose = verbose
        
        self.estimators_ = None
        
    def _estimator_(self, X, y):
    
        warnings.simplefilter(action='ignore', category=FutureWarning)
        
        if (y.name[2] == 'SCHOOL AND OFFICE SUPPLIES') | (y.name[2] == 'LIQUOR,WINE,BEER'):
 #       if (y.name[2] == 'SCHOOL AND OFFICE SUPPLIES'):
            
            model = RandomForestRegressor(n_estimators = 300, n_jobs=-1, random_state=1)
            
        else:
            
            model = Ridge(fit_intercept=True, solver='auto', alpha=0.5, normalize=True)
            
        model.fit(X, y)

        return model

    def fit(self, X, y):

        self.estimators_ = Parallel(n_jobs=self.n_jobs, 
                              verbose=self.verbose,
                              )(delayed(self._estimator_)(X, y.iloc[:, i]) for i in range(y.shape[1]))
        
        return
    
    def predict(self, X):
        
        y_pred = Parallel(n_jobs=self.n_jobs, 
                          verbose=self.verbose)(delayed(e.predict)(X) for e in self.estimators_)
        
        return np.stack(y_pred, axis=1)

In [278]:
model = CustomRegressor(n_jobs=-1, verbose=0)
model.fit(Xfinal, y)
y_pred = pd.DataFrame(model.predict(Xfinal), index=Xfinal.index, columns=y.columns)

In [279]:
y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()
y_target['sales_pred'] = y_pred['sales'].clip(0.) 
y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred']))

family
AUTOMOTIVE                   0.24
BABY CARE                    0.06
BEAUTY                       0.24
BEVERAGES                    0.18
BOOKS                        0.02
BREAD/BAKERY                 0.12
CELEBRATION                  0.28
CLEANING                     0.19
DAIRY                        0.13
DELI                         0.10
EGGS                         0.14
FROZEN FOODS                 0.13
GROCERY I                    0.20
GROCERY II                   0.33
HARDWARE                     0.25
HOME AND KITCHEN I           0.24
HOME AND KITCHEN II          0.20
HOME APPLIANCES              0.14
HOME CARE                    0.11
LADIESWEAR                   0.24
LAWN AND GARDEN              0.20
LINGERIE                     0.37
LIQUOR,WINE,BEER             0.26
MAGAZINES                    0.23
MEATS                        0.11
PERSONAL CARE                0.12
PET SUPPLIES                 0.20
PLAYERS AND ELECTRONICS      0.20
POULTRY                      0.11
PREPARE

In [280]:
y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])).mean()

0.17988581196407225

In [281]:
end_test='2017-08-31'
start_test='2017-08-16'
X_test = dp.out_of_sample(steps=16)


X_test = X_test.join(calendar)
test.date = test.date.dt.to_period('D')
test = test.set_index(['date'])
X_test['promotion'] = test[['onpromotion']].groupby('date').sum()

sales_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
sales_pred = sales_pred.stack(['store_nbr', 'family'])
sales_pred[sales_pred < 0] = 0. 

In [282]:
y.columns

MultiIndex([('sales', '1',                 'AUTOMOTIVE'),
            ('sales', '1',                  'BABY CARE'),
            ('sales', '1',                     'BEAUTY'),
            ('sales', '1',                  'BEVERAGES'),
            ('sales', '1',                      'BOOKS'),
            ('sales', '1',               'BREAD/BAKERY'),
            ('sales', '1',                'CELEBRATION'),
            ('sales', '1',                   'CLEANING'),
            ('sales', '1',                      'DAIRY'),
            ('sales', '1',                       'DELI'),
            ...
            ('sales', '9',                  'MAGAZINES'),
            ('sales', '9',                      'MEATS'),
            ('sales', '9',              'PERSONAL CARE'),
            ('sales', '9',               'PET SUPPLIES'),
            ('sales', '9',    'PLAYERS AND ELECTRONICS'),
            ('sales', '9',                    'POULTRY'),
            ('sales', '9',             'PREPARED FOODS')

In [283]:
My_submission = pd.read_csv('data/sample_submission.csv', index_col='id')
My_submission.sales = sales_pred.values
My_submission.to_csv('submissioncomp.csv', index=True)

In [284]:
My_submission

,sales
id,
3000888,4.45
3000889,0.00
3000890,4.08
3000891,2352.09
3000892,0.35
...,...
3029395,296.45
3029396,87.33
3029397,1210.63
